### THUC HIEN TREN GG COLAB

Cai dat thu vien

In [ ]:
!pip uninstall torchtext torch -y
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 -f https://download.pytorch.org/whl/cu118/torch_stable.html
!pip install torchtext==0.15.2 --no-cache-dir
!pip install timm

Giai nen file zip

In [ ]:
!unzip vqa_coco_dataset.zip

In [ ]:
import torch
import torch.nn as nn
import torchtext
import os
import random
import numpy as np
import pandas as pd
import spacy
import timm
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchvision import transforms

cai dat gia tri ngau nhien co dinh

In [ ]:
def set_seed (seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
seed = 59
set_seed(seed)


chia bo du lieu train val test

In [ ]:
# Load train data
train_data = []
train_set_path = './vaq2.0.TrainImages.txt'

with open (train_set_path, "r") as f:
  lines = f.readlines()
  for line in lines:
    temp = line.split('\t')
    qa = temp[1].split('?')
    if len(qa) == 3:
      answer = qa[2].strip()
    else:
      answer = qa[1].strip()

    data_sample = {
      'image_path': temp[0][: -2],
      'question': qa[0] + '?',
      'answer': answer
    }
    train_data.append(data_sample)

# Load val data
val_data = []
val_set_path = './vaq2.0.DevImages.txt'

with open (val_set_path, "r") as f:
  lines = f.readlines()
  for line in lines:
    temp = line.split('\t')
    qa = temp[1].split('?')
    if len(qa) == 3:
      answer = qa[2].strip()
    else:
      answer = qa[1].strip()

    data_sample = {
      'image_path': temp[0][: -2],
      'question': qa[0] + '?',
      'answer': answer
    }
    val_data.append(data_sample)

# Load test data
test_data = []
test_set_path = './vaq2.0.TestImages.txt'

with open (test_set_path, "r") as f:
  lines = f.readlines()
  for line in lines:
    temp = line.split('\t')
    qa = temp[1].split('?')
    if len(qa) == 3:
      answer = qa[2].strip()
    else:
      answer = qa[1].strip()

    data_sample = {
      'image_path': temp[0][: -2],
      'question': qa[0] + '?',
      'answer': answer
    }
    test_data.append(data_sample)



Xay dung bo tu vung

In [ ]:
eng = spacy.load("en_core_web_sm")

def get_tokens(data_iter):
  for sample in data_iter:
    question = sample['question']
    yield[token.text for token in eng.tokenizer(question)]

vocab = build_vocab_from_iterator(
  get_tokens(train_data),
  min_freq=2,
  specials = ['<pad>', '<sos>', '<eos >', '<unk>'],
  special_first=True
)
vocab.set_default_index(vocab['<unk>'])

Xay dung dictionary mapping classes

In [ ]:
classes = set([sample ['answer'] for sample in train_data])
label2idx = {
  cls_name: idx for idx, cls_name in enumerate(classes)
}

idx2label = {
  idx: cls_name for idx, cls_name in enumerate(classes)
}

Xay dung ham tokenize

In [ ]:
def tokenize (question, max_seq_len):
  tokens = [token.text for token in eng.tokenizer(question)]
  sequence = [vocab[token] for token in tokens]
  if len(sequence) < max_seq_len:
    sequence += [vocab['<pad>']] * (max_seq_len - len(sequence))
  else:
    sequence =sequence[:max_seq_len]

  return sequence

Xay dung class Pytorch dataset

In [ ]:
class VQADataset (Dataset):
  def __init__(
    self,
    data,
    label2idx,
    max_seq_len =20,
    transform=None,
    img_dir='val2014-resised/'
  ):
    self.transform = transform
    self.data = data
    self.max_seq_len = max_seq_len
    self.img_dir = img_dir
    self.label2idx = label2idx

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    img_path = os.path.join(self.img_dir, self.data[index]['image_path'])
    img = Image.open(img_path).convert('RGB')
    if self.transform:
      img = self.transform (img)
    question = self.data[index]['question']
    question = tokenize(question, self.max_seq_len)
    question = torch.tensor(question, dtype=torch.long)
    label = self.data[index]['answer']
    label = label2idx[label]
    label = torch. tensor(label, dtype=torch. long)
    return img, question, label


Xay dung Pytorch transform

In [ ]:
data_transform = {
  'train': transforms.Compose ([
    transforms.Resize(size=(224, 224)),
    transforms.CenterCrop(size=180),
    transforms.ColorJitter(brightness =0.1, contrast =0.1, saturation=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(3),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
  ]),
  'val': transforms.Compose ([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
  ])
}

Khai bao dataset object cho ba bo train val test

In [ ]:
train_dataset = VQADataset(
  train_data,
  label2idx=label2idx,
  transform=data_transform['train']
)
val_dataset = VQADataset(
  val_data,
  label2idx=label2idx,
  transform=data_transform['val']
)
test_dataset = VQADataset(
  test_data,
  label2idx=label2idx,
  transform=data_transform['val']
)

Khai bao DataLoader

In [ ]:
train_batch_size = 256
test_batch_size = 32

train_loader = DataLoader (
  train_dataset,
  batch_size=train_batch_size,
  shuffle=True
)
val_loader = DataLoader (
  val_dataset,
  batch_size=test_batch_size,
  shuffle=False
)
test_loader = DataLoader (
  test_dataset,
  batch_size=test_batch_size,
  shuffle=False
)

Xay dung model

In [ ]:
class VQAModel(nn.Module):
  def __init__ (
    self,
    n_classes,
    img_model_name,
    embeddding_dim,
    n_layers=2,
    hidden_size=256,
    drop_p=0.2
  ):
    super(VQAModel, self).__init__()
    self.image_encoder = timm.create_model(
      img_model_name,
      pretrained=True,
      num_classes = hidden_size
    )
    for param in self.image_encoder.parameters():
      param.requires_grad = True

    self.embedding = nn.Embedding(len(vocab), embeddding_dim)
    self.lstm1 = nn.LSTM(
        input_size=embeddding_dim,
        hidden_size=hidden_size,
        num_layers=n_layers,
        batch_first=True,
        bidirectional=True,
        dropout=drop_p
    )

    self.fc1 = nn.Linear(hidden_size * 3, hidden_size)
    self.dropout = nn.Dropout(drop_p)
    self.gelu = nn.GELU()
    self.fc2 = nn.Linear(hidden_size, n_classes)

  def forward(self, img, text):
    img_features = self.image_encoder(img)

    text_emb = self.embedding(text)
    lstm_out, _ = self.lstm1(text_emb)

    lstm_out = lstm_out[:, -1, :]

    combined = torch.cat((img_features, lstm_out), dim=1)
    x = self.fc1(combined)
    x = self.gelu(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

n_classes = len(classes)
img_model_name = 'resnet18'
hidden_size = 256
n_layers = 2
embeddding_dim = 128
drop_p = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = VQAModel(
  n_classes=n_classes,
  img_model_name=img_model_name,
  embeddding_dim=embeddding_dim,
  n_layers=n_layers,
  hidden_size=hidden_size,
  drop_p=drop_p
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Xay dung ham train va evaluate

In [ ]:
def evaluate(model, dataloader, criterion, device):
  model.eval()
  correct = 0
  total = 0
  losses = []
  with torch.no_grad():
    for image, question, labels in dataloader:
      image, question, labels =image.to(device), question.to(device), labels.to(device)
      outputs = model(image, question)
      loss = criterion(outputs, labels)
      losses. append(loss.item())
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  loss = sum (losses) / len (losses)
  acc = correct / total

  return loss, acc
def fit (
  model,
  train_loader,
  val_loader,
  criterion,
  optimizer,
  scheduler,
  device,
  epochs
):
  train_losses = []
  val_losses = []

  for epoch in range (epochs):
    batch_train_losses = []

    model.train()
    for idx, (images, questions, labels) in enumerate( train_loader):
      images = images.to(device)
      questions = questions.to(device)
      labels = labels.to (device)

      optimizer.zero_grad()
      outputs = model(images, questions)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      batch_train_losses.append(loss.item())

    train_loss = sum(batch_train_losses) / len(batch_train_losses)
    train_losses.append(train_loss)

    val_loss, val_acc = evaluate(
      model, val_loader,
      criterion, device
    )
    val_losses.append(val_loss)

    print (f'EPOCH {epoch + 1}: \tTrain loss: {train_loss:.4f}\tVal loss: {val_loss:.4f}\tVal Acc: {val_acc}')
    scheduler.step()

  return train_losses, val_losses



Khai bao ham loss optimizer scheduler

In [ ]:
lr = 1e-3
epochs = 50
scheduler_step_size = epochs * 0.8

criterion = nn.CrossEntropyLoss ()

optimizer = torch.optim.Adam(
  model.parameters(),
  lr=lr
)

scheduler = torch.optim.lr_scheduler.StepLR (
  optimizer,
  step_size=scheduler_step_size,
  gamma=0.1
)

Training

In [ ]:
train_losses, val_losses = fit(
  model,
  train_loader,
  val_loader,
  criterion,
  optimizer,
  scheduler,
  device,
  epochs
)

Evaluation

In [ ]:
val_loss, val_acc = evaluate(
  model,
  val_loader,
  criterion,
  device
)
test_loss, test_acc = evaluate(
  model,
  test_loader,
  criterion,
  device
)

print('Evaluation on val/test dataset')
print('Val accuracy: ' val_acc)
print('Test accuracy: ', test_acc)

## Evaluation results from author
# Val accuracy: 0.4989754098360656
# Test accuracy: 0.5084075173095944
